In [1]:
import pandas as pd, numpy as np

In [2]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')

In [3]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [4]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]

In [5]:
dc=dc.set_index('uid')
dm=dm.set_index('uid')

In [6]:
dmi=dm.loc[dc.index.dropna()]

In [7]:
df=dmi.merge(dc).drop_duplicates()

Working df ready. Now infer location.

In [8]:
locs=df[['category','icategory','tags','title','content']]

In [9]:
locs

,category,icategory,tags,title,content
0,aktualis,agerpres,"['medve', 'Udvarhelyszék']","Videóra vették, ahogy elhurcolja a megölt tehe...",Három nap leforgása alatt három tehenet öltek ...
1,aktualis,csik,"['Háromszék', 'medve']",Már a háromszéki prefektusnak is nagyon elege ...,Napról napra nagyobb gondot jelentenek a medvé...
2,aktualis,haromszek,"['Háromszék', 'medve']",Kútba esett egy medve Bibarcfalván,Beleesett egy medve a kútba az erdővidéki Biba...
3,aktualis,csik,"['medve', 'Székelyudvarhely']",Székelyudvarhely utcáin portyázott a bocsos medve,"Videó is készült arról a bocsos anyamedvéről, ..."
4,aktualis,udvarhely,"['Homoródalmás', 'medve']",Letörte a csűr ajtaját is a medve a juhokért,Csűrbe zárta estére a megmaradt jószágait a ho...
5,vilag,udvarhely,"['Belföld', 'medve']",Medve támadt egy lengyel turistára,Lengyel turistára támadt egy medve a Radnai-ha...
6,aktualis,csik,"['Háromszék', 'medve']",Amikor négy tehén és három medve néz farkassze...,"Azt, hogy leggyakrabban miként ér véget a medv..."
7,aktualis,csik,"['Csíksomlyó', 'medve']",Hárombocsos anyamedve közelítette meg a csíkso...,Egy hárombocsos anyamedve közelítette meg a Xa...
8,aktualis,csik,"['Maros megye', 'medve']","Halálra gázoltak egy medvét, majd az úttesten ...",Elütöttek és az úton hagytak egy medvét kedden...
9,aktualis,agerpres,"['Csíksomlyó', 'medve']",RO-Alert: Csíksomlyón jár a medve,A RO-Alert rendszeren keresztül figyelmeztette...


In [10]:
import json

In [11]:
lak=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [12]:
laks=[i for i in megyek.keys() if ((megyek[i]!='Más megye')and(i in lak.keys()))]

In [13]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [14]:
#look for perfect matches
decoded_locs={}
min_loc_len=5
for i in locs.index:
    if '[' in locs.loc[i]['tags']:
        k=locs.loc[i]['tags'].replace("'",'"')
        for j in json.loads(k):
            if j in laks:
                if i not in decoded_locs:decoded_locs[i]=[]
                decoded_locs[i].append(j)
                print('1',k,j)
    for j in locs.loc[i]['title'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('2',j)
    for j in locs.loc[i]['content'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('3',j)
    for j in locs.loc[i]['title'].split():
        for k in laks:
            if ((len(j)>min_loc_len) and (len(k)>min_loc_len)):
                if levenshteinDistance(j,k)<3:
                    if i not in decoded_locs:decoded_locs[i]=[]
                    decoded_locs[i].append(k)
                    print('4',j,k)
    for j in locs.loc[i]['content'].split():
        for k in laks:
            if ((len(j)>min_loc_len) and (len(k)>min_loc_len)):
                if levenshteinDistance(j,k)<3:
                    if i not in decoded_locs:decoded_locs[i]=[]
                    decoded_locs[i].append(k)
                    print('5',j,k)

3 Pálpataka
5 Pálpataka Pálpataka
5 Pálpataka Pálpatak
3 Kovászna
5 Kovászna Kovászna
4 Bibarcfalván Bibarcfalva
5 Bibarcfalván Bibarcfalva
1 ["medve", "Székelyudvarhely"] Székelyudvarhely
2 Székelyudvarhely
3 Székelyudvarhely
4 Székelyudvarhely Székelyudvarhely
5 Szabók Szabéd
5 Székelyudvarhely Székelyudvarhely
1 ["Homoródalmás", "medve"] Homoródalmás
5 homoródalmási Homoródalmás
1 ["Csíksomlyó", "medve"] Csíksomlyó
4 csíksomlyói Csíksomlyó
5 csíksomlyói Csíksomlyó
3 Körtvélyfája
3 Petele
3 Szászrégen
5 Körtvélyfája Körtvélyfája
5 Petele Kebele
5 Petele Petele
5 Szászrégen Szászrégen
1 ["Csíksomlyó", "medve"] Csíksomlyó
4 Csíksomlyón Csíksomlyó
5 Csíksomlyón Csíksomlyó
3 Nagyborosnyó
4 Nagyborosnyón Nagyborosnyó
5 Nagyborosnyón Nagyborosnyó
5 Nagyborosnyó Nagyborosnyó
1 ["medve", "Tusnádfürdő"] Tusnádfürdő
5 Tusnádfürdőn: Tusnádfürdő


In [22]:
predicted_locs=[]
for i in range(len(locs.index)):
    if locs.index[i] in decoded_locs: 
        dummy=list(set(decoded_locs[locs.index[i]]))
        if len(dummy)==1:
            dummy=dummy[0]
        else:
            dummy=repr(list(set(decoded_locs[locs.index[i]])))
        predicted_locs.append(dummy)
    else: 
        predicted_locs.append('')

In [23]:
locs['predicted_locs']=predicted_locs
locs['actual_locs']=predicted_locs

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
lm=locs[['title','content','predicted_locs','actual_locs']]

In [26]:
lm.to_excel('data/szekelyhon_locs.xlsx')
lm.to_excel('data/szekelyhon_locs_curated.xlsx')

Open `data/szekelyhon_locs_curated` and manually verify predicted locations. Amend `actual_locs` where necessary.